# Example of DOV search methods for quartaire stratigrafie

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_quartaire_stratigrafie.ipynb)

## Use cases:
* Select records in a bbox
* Select records in a bbox with selected properties
* Select records within a distance from a point
* Select records in a municipality
* Get records using info from wfs fields, not available in the standard output dataframe

In [1]:
%matplotlib inline
import os, sys
import inspect

In [2]:
# check pydov path
import pydov

## Get information about code base

In [3]:
from pydov.search.interpretaties import QuartairStratigrafieSearch
# information about the QuartairStratigrafie type (In Dutch):
ip_quart = QuartairStratigrafieSearch()
ip_quart.get_description()

'De quartaire afzettingen in Vlaanderen, de jongste afzettingen, zijn lateraal en verticaal heterogeen en varieren erg in dikte. De afzettingen zijn vooral onder continentale omstandigheden tot stand gekomen (rivier-, duinafzettingen). Eigenlijk zijn het pakketten die zelden meer dan 30 meter en soms zelfs minder dan 1 meter dik zijn. Omwille van deze grote eigenheid en de grote mate van detail van de beschrijvingen, werd hiervoor een eigen stratigrafische codering uitgewerkt voor Vlaanderen.'

In [4]:
# information about the available fields for a QuartairStratigrafie object
fields = ip_quart.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
start_interpretatie_mtaw
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
eerste_invoer
geom
diepte_laag_van
diepte_laag_tot
lid1
relatie_lid1_lid2
lid2


In [5]:
# print information for a certain field
fields['Type_proef']

{'name': 'Type_proef', 'definition': "Het type van de proef waarvan de quartaire stratigrafie een interpretatie is. Bij een quartaire stratigrafie zal dit altijd 'Boring' zijn.", 'type': 'string', 'list': False, 'notnull': False, 'query': True, 'cost': 1, 'codelist': <pydov.util.codelists.AbstractCodeList: <pydov.util.codelists.CodeListItem: code: Boring, label: Boring, definition: De quartaire stratigrafie is een interpretatie van een boring.>>}

The cost is an arbitrary attribute to indicate if the information is retrieved from a wfs query (cost = 1), 
or from an xml (cost = 10)

Alternatively, you can list all the fields and their details by inspecting the `get_fields()` output or the search instance itself in a notebook:

In [6]:
ip_quart

## Try-out of use cases

### Select interpretations in a bbox

In [7]:
from pydov.util.location import Within, Box

# Get all interpretations in a bounding box (llx, lly, ulx, uly)
# the pkey_boring link is not available below, but is in the df
df = ip_quart.search(location=Within(Box(132815, 191700, 133815, 192700)),
                     max_features=10)
df.head()

[000/001] .
[000/003] ...


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132949.0,192068.0,2.57,0.0,4.5,K,T,K
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132949.0,192068.0,2.57,4.5,9.5,F,T,F
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132949.0,192068.0,2.57,9.5,10.0,Rf,T,Rf
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132975.0,191969.0,2.84,0.0,4.5,K,T,K
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132975.0,191969.0,2.84,4.5,7.5,F,T,F


### Select interpretations in a bbox with selected properties

In [8]:
# list available query methods
methods = [i for i,j in inspect.getmembers(sys.modules['owslib.fes2'], 
                                           inspect.isclass) 
           if 'Property' in i]
methods

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In [9]:
from owslib.fes2 import PropertyIsGreaterThanOrEqualTo

The property feature methodes listed above are available from the owslib module. These were not adapted for use in pydov.

In [10]:
# Get deep boreholes in a bounding box 
from owslib.fes2 import PropertyIsLike
# the propertyname can be any of the fields of the interpretations object that belong to the wfs source
# the literal is always a string, no matter what its definition is in the boring object (string, float...)
query = PropertyIsLike(
        propertyname='betrouwbaarheid_interpretatie', literal='onbekend')
df = ip_quart.search(location=Within(Box(132815, 191700, 133815, 192700)),
                     query=query
                    )
df.head()

[000/001] .
[000/003] ccc


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132949.0,192068.0,2.57,0.0,4.5,K,T,K
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132949.0,192068.0,2.57,4.5,9.5,F,T,F
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132949.0,192068.0,2.57,9.5,10.0,Rf,T,Rf
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132975.0,191969.0,2.84,0.0,4.5,K,T,K
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1974...,onbekend,132975.0,191969.0,2.84,4.5,7.5,F,T,F


### Select records within a distance from a point

In [11]:
from pydov.util.location import Point, WithinDistance

# Get all interpretations within a defined distance from a point
location = WithinDistance(location=Point(133000, 192000),
                          distance=1000, distance_unit='meter')
df = ip_quart.search(location=location)
df.head()

[000/001] .
[000/015] .....cc.c......


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1943...,onbekend,132066.0,191830.0,3.15,0.0,2.0,K,T,K
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1943...,onbekend,132066.0,191830.0,3.15,2.0,3.0,D,T,D
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1943...,onbekend,132066.0,191830.0,3.15,3.0,11.5,F,T,Rf
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1943...,onbekend,132066.0,191830.0,3.15,11.5,30.0,$,T,$
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1943...,onbekend,132017.0,191843.0,3.16,0.0,1.5,K,T,K


### Select interpretations in a municipality

In [12]:
from owslib.fes2 import PropertyIsEqualTo
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Hamme')
df = ip_quart.search(query=query)
df.head()

[000/001] .
[000/024] ........................


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,lid1,relatie_lid1_lid2,lid2
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1894...,goed,129770.0,198680.0,4.60,0.00,3.50,F,T,F
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1909...,onbekend,136049.6,192883.4,3.00,0.00,0.40,!,T,!
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1909...,onbekend,136049.6,192883.4,3.00,0.40,11.18,ô,T,D
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1909...,onbekend,136049.6,192883.4,3.00,11.18,33.00,$,T,$
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1972...,onbekend,133211.7,196346.1,4.63,2.00,2.19,D,T,D


### Visualize results

Using Folium, we can display the results of our search on a map.

In [13]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer
# for windows 10 users, we could not get it working with Microsoft Edge
# change the browser to Chrome in the Notebook config file instead

In [14]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [15]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['lid1'][loc]).add_to(marker_cluster)
fmap